In [59]:
pip install requests

In [60]:
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd

In [61]:
def get_api_response(url, headers, params):
    try:
        response = requests.request("GET", url, headers=headers, params=params)
        response.raise_for_status()
    
    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.InvalidHeader as erri:
        return "A Header Error occurred: " + repr(erri)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)
        
    return response.json()


In [62]:
stock=input()
print("The Stock we will research is: " + stock)
apikey="fnu7m7Uigl2tqyCspHnLs4dAPIgph52M7zbHjUlL"

url = "https://yfapi.net/v6/finance/quote"
url2 = "https://yfapi.net/v11/finance/quoteSummary/AAPL?lang=en&region=US&modules=financialData%2CbalanceSheetHistory"
querystring = {"symbols":stock}

headers = {
    'x-api-key': apikey
    }

response = get_api_response(url, headers, querystring)
response2 = get_api_response(url2, headers, querystring)


The Stock we will research is: AAPL


In [63]:
# Ticker, Full Name of the Stock, Current Price, Target Mean Price, Cash on Hand from the last balance sheet, Gross Profits

stock_json = response
stock_json2 = response2
try:
  long_name = stock_json['quoteResponse']['result'][0]["longName"]
  regular_market_price = stock_json['quoteResponse']['result'][0]["regularMarketPrice"]
  target_mean_price = stock_json2["quoteSummary"]["result"][0]["financialData"]["targetMeanPrice"]["fmt"]
  cash_on_hand = stock_json2['quoteSummary']['result'][0]["balanceSheetHistory"]["balanceSheetStatements"][0]["cash"]["fmt"]
  gross_profits = stock_json2["quoteSummary"]["result"][0]["financialData"]["grossProfits"]["fmt"]

  print(stock + "\n" + long_name + "\n" + "Price: $" + str(regular_market_price) + "\n" + "Target Mean Price: $" + str(target_mean_price) + "\n"
        + "Cash on Hand from Last Balance Sheet: " + str(cash_on_hand) + "\n" + "Gross Profits: " + str(gross_profits))
  
except KeyError:
  print("Error Occured")
except IndexError:
  print("Error occured")
except Exception:
  print("Error occured")


AAPL
Apple Inc.
Price: $170.43
Target Mean Price: $188.43
Cash on Hand from Last Balance Sheet: 23.65B
Gross Profits: 170.78B


In [64]:
data = {
    "Stock": [stock],
    "Name": [long_name],
    "Price": [regular_market_price],
    "Target Mean Price": [target_mean_price],
    "Cash on Hand from Last Balance Sheet": [cash_on_hand],
    "Gross Profits": [gross_profits]
}

df = pd.DataFrame(data)

json = 'stock_data.json'
df.to_json(json, orient='records')

csv = 'stock_data.csv'
df.to_csv(csv, index=False)